In [3]:
import os
import sys
import random
import re
import pandas as pd
from Bio import SeqIO
from pathlib import Path

In [5]:
file_prefix = 'tair10'
Path("./result/sim_" + file_prefix +"_genome").mkdir(parents=True, exist_ok=True)

In [98]:
rm_outsum = "/workspace/cflthc/script/KRIP_TE/10_TEgenomeSimulator/model_species_RM/tair10/AthaGenome.fa.out.summary"

In [99]:
te_sum = pd.read_table(rm_outsum, sep='\t')

In [100]:
te_sum['subclass'] = te_sum['superfamily'].replace("/.*", "", regex=True)

In [101]:
te_superfamily = list(te_sum['superfamily'])
# tsd (use prior knowledge)
ltr_retro = ['LTR/Copia', 'LTR/Gypsy', 'LTR/Ty3', 'LTR/Solo', 'LTR/unknown'] 
line = ['LINE/unknown', 'LINE/L1', 'LINE/LINE_Unknown']
sine = ['SINE/unknown', 'SINE/tRNA', 'SINE/SINE_Unknown']
DTA = ['DNA/hAT', 'DNAnona/hAT', 'DNAauto/hAT', 'DNA/DTA']
DTC = ['DNA/CACTA', 'DNAnona/CACTA', 'DNAauto/CACTA', 'DNA/DTC']
DTH = ['DNA/Harbinger', 'DNA/DTH']
DTM = ['DNA/MuDR', 'DNAnona/MULE', 'DNAauto/MULE', 'DNA/DTM']
DTT = ['DNA/Mariner', 'DNA/DTT']
helitron = ['DNA/Helitron', 'DNAnona/Helitron', 'DNAauto/Helitron', 'RC/Helitron']
mite = ['MITE/Stow', 'MITE/Tourist']

tsd = []
for sup_fam in te_superfamily:
    if sup_fam in ltr_retro:
        tsd_range = '5,5'
    elif sup_fam in line:
        tsd_range = '5,20'
    elif sup_fam in sine:
        tsd_range = '5,20'
    elif sup_fam in DTA:
        tsd_range = '5,8'
    elif sup_fam in DTC:
        tsd_range = '2,4'
    elif sup_fam in DTH:
        tsd_range = '3,3'
    elif sup_fam in DTM:
        tsd_range = '8,9'
    elif sup_fam in DTT:
        tsd_range = '2,2'
    elif sup_fam in helitron:
        tsd_range = '0,0'
    elif sup_fam in mite:
        tsd_range = '2,10'
    else:
        tsd_range = '0,0' 
    tsd.append(str(tsd_range))

In [102]:
te_sum['idn'] = round(te_sum['mean_identity'] * 100, 0)
te_sum['idn'] = te_sum['idn'].astype(int)

In [103]:
te_sum['sd'] = round(te_sum['sd_identity'] * 100, 0)
te_sum['sd'] = te_sum['sd'].astype(int)

In [104]:
te_sum['indels'] = round(te_sum['mean_indel'] * 100, 0)
te_sum['indels'] = te_sum['indels'].astype(int)

In [107]:
# adjust the propostion of fragmented loci
te_sum['new_frag'] = (te_sum['fragment_ratio'] + 1) / 2
te_sum['frag'] = round(te_sum['new_frag'] * 100, 0)
te_sum['frag'] = te_sum['frag'].astype(int)

In [109]:
te_sum['tsd'] = pd.DataFrame({'tsd': tsd})

In [110]:
# nest TE loci (as a proportion to total TE loci of the family; only apply for Copia and Gypsy)
copia_gypsy = ['LTR/Copia', 'LTR/Gypsy', 'LTR/Ty3'] 

nested = []
minimum = 0
maximum = 30
for sup_fam in te_superfamily:
    if sup_fam in copia_gypsy:
        nest_value = random.choice(range(minimum, maximum + 1))
    else:
        nest_value = int(0)
    nested.append(nest_value)

In [111]:
te_sum['nest'] = pd.DataFrame({'nest': nested})

In [112]:
te_sum

,family,superfamily,full_length_bp,copynumber,mean_identity,sd_identity,mean_indel,total_length,mean_length,mean_headcut,...,fragment_ratio,original_family,subclass,idn,sd,indels,tsd,new_frag,frag,nest
0,ATDNA12T3_2,DNA,828,605,0.820744,0.011902,0.037970,70742,116.9290,192.54500,...,0.983471,ATDNA12T3_2#DNA,DNA,82,1,4,"0,0",0.991735,99,0
1,ATDNA12T3A,DNA,2372,38,0.734263,0.030378,0.237828,13864,364.8420,978.78900,...,1.000000,ATDNA12T3A#DNA,DNA,73,3,24,"0,0",1.000000,100,0
2,ATREP18,DNA,1791,573,0.737901,0.006513,0.260353,124000,216.4050,1190.44000,...,0.998255,ATREP18#DNA,DNA,74,1,26,"0,0",0.999127,100,0
3,ATREP19,DNA,587,7,0.786571,0.070517,0.398692,1823,260.4290,40.85710,...,0.857143,ATREP19#DNA,DNA,79,7,40,"0,0",0.928571,93,0
4,ATTIR16T3A,DNA,501,135,0.808874,0.016170,0.302820,39682,293.9410,89.76300,...,0.748148,ATTIR16T3A#DNA,DNA,81,2,30,"0,0",0.874074,87,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,ATENSAT1,Unspecified,1949,36,0.882611,0.062935,0.261903,45258,1257.1700,364.63900,...,0.388889,ATENSAT1,Unspecified,88,6,26,"0,0",0.694445,69,0
322,ATENSAT2,Unspecified,974,26,0.885538,0.048939,0.234769,16444,632.4620,61.50000,...,0.538462,ATENSAT2,Unspecified,89,5,23,"0,0",0.769231,77,0
323,ATMSAT1,Unspecified,97,346,0.922939,0.011663,0.027067,32846,94.9306,1.04335,...,0.063584,ATMSAT1,Unspecified,92,1,3,"0,0",0.531792,53,0
324,ATREPX1,Unspecified,2434,150,0.762580,0.016377,0.267672,55611,370.7400,882.56000,...,0.993333,ATREPX1,Unspecified,76,2,27,"0,0",0.996667,100,0


In [ ]:
#TE_family	subclass	superfamily	count	idn	sd	indels	tsd	length	frag	nest

In [113]:
df = te_sum[['original_family', 'subclass', 'superfamily', 'copynumber', 'idn', 'sd', 'indels', 'tsd', 'full_length_bp', 'frag', 'nest']]

In [124]:
df = df.rename(columns={'original_family': '#TE_family', 'copynumber': 'count', 'full_length_bp': 'length'})

In [125]:
df

,#TE_family,subclass,superfamily,count,idn,sd,indels,tsd,length,frag,nest
0,ATDNA12T3_2#DNA,DNA,DNA,605,82,1,4,"0,0",828,99,0
1,ATDNA12T3A#DNA,DNA,DNA,38,73,3,24,"0,0",2372,100,0
2,ATREP18#DNA,DNA,DNA,573,74,1,26,"0,0",1791,100,0
3,ATREP19#DNA,DNA,DNA,7,79,7,40,"0,0",587,93,0
4,ATTIR16T3A#DNA,DNA,DNA,135,81,2,30,"0,0",501,87,0
...,...,...,...,...,...,...,...,...,...,...,...
321,ATENSAT1,Unspecified,Unspecified,36,88,6,26,"0,0",1949,69,0
322,ATENSAT2,Unspecified,Unspecified,26,89,5,23,"0,0",974,77,0
323,ATMSAT1,Unspecified,Unspecified,346,92,1,3,"0,0",97,53,0
324,ATREPX1,Unspecified,Unspecified,150,76,2,27,"0,0",2434,100,0


In [119]:
df['idn'] * 1.5

0      123.0
1      109.5
2      111.0
3      118.5
4      121.5
       ...  
321    132.0
322    133.5
323    138.0
324    114.0
325    117.0
Name: idn, Length: 326, dtype: float64

In [92]:
te_sum

,family,superfamily,full_length_bp,copynumber,mean_identity,sd_identity,mean_indel,total_length,mean_length,mean_headcut,mean_tailcut,mean_cut,fragment_copy,fragment_ratio,original_family,subclass,tsd,new_frag,nest,idn
0,ATDNA12T3_2,DNA,828,605,0.820744,0.011902,0.037970,70742,116.9290,192.54500,518.478000,711.02300,595.0,0.983471,ATDNA12T3_2#DNA,DNA,"0,0",0.991735,0,82
1,ATDNA12T3A,DNA,2372,38,0.734263,0.030378,0.237828,13864,364.8420,978.78900,1023.630000,2002.42000,38.0,1.000000,ATDNA12T3A#DNA,DNA,"0,0",1.000000,0,73
2,ATREP18,DNA,1791,573,0.737901,0.006513,0.260353,124000,216.4050,1190.44000,379.550000,1569.99000,572.0,0.998255,ATREP18#DNA,DNA,"0,0",0.999127,0,74
3,ATREP19,DNA,587,7,0.786571,0.070517,0.398692,1823,260.4290,40.85710,286.714000,327.57100,6.0,0.857143,ATREP19#DNA,DNA,"0,0",0.928571,0,79
4,ATTIR16T3A,DNA,501,135,0.808874,0.016170,0.302820,39682,293.9410,89.76300,104.030000,193.79300,101.0,0.748148,ATTIR16T3A#DNA,DNA,"0,0",0.874074,0,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,ATENSAT1,Unspecified,1949,36,0.882611,0.062935,0.261903,45258,1257.1700,364.63900,292.806000,657.44400,14.0,0.388889,ATENSAT1,Unspecified,"0,0",0.694445,0,88
322,ATENSAT2,Unspecified,974,26,0.885538,0.048939,0.234769,16444,632.4620,61.50000,269.808000,331.30800,14.0,0.538462,ATENSAT2,Unspecified,"0,0",0.769231,0,89
323,ATMSAT1,Unspecified,97,346,0.922939,0.011663,0.027067,32846,94.9306,1.04335,0.965318,2.00867,22.0,0.063584,ATMSAT1,Unspecified,"0,0",0.531792,0,92
324,ATREPX1,Unspecified,2434,150,0.762580,0.016377,0.267672,55611,370.7400,882.56000,1123.200000,2005.76000,149.0,0.993333,ATREPX1,Unspecified,"0,0",0.996667,0,76


In [74]:
x = round(5.76543, 2)
print(x)

5.77


In [69]:
te_sum.sample(n=10)

,family,superfamily,full_length_bp,copynumber,mean_identity,sd_identity,mean_indel,total_length,mean_length,mean_headcut,mean_tailcut,mean_cut,fragment_copy,fragment_ratio,original_family,subclass,tsd,new_frag,nest
131,ATLINE1_6,LINE/L1,5525,164,0.642616,0.008091,0.149582,128366,782.720,2779.1000,1948.2900,4727.3900,164.0,1.000000,ATLINE1_6#LINE/L1,LINE,"5,20",1.000000,0
225,ATCOPIA84,LTR/Copia,4882,13,0.821538,0.030935,0.168076,5676,436.615,3263.3100,1181.4600,4444.7700,12.0,0.923077,ATCOPIA84#LTR/Copia,LTR,"5,5",0.961539,12
200,ATCOPIA61,LTR/Copia,5019,8,0.816750,0.054005,0.200987,6938,867.250,1448.0000,2711.6200,4159.6200,7.0,0.875000,ATCOPIA61#LTR/Copia,LTR,"5,5",0.937500,22
223,ATCOPIA82,LTR/Copia,5295,6,0.827833,0.064843,0.050037,6566,1094.330,903.8330,3298.8300,4202.6700,5.0,0.833333,ATCOPIA82#LTR/Copia,LTR,"5,5",0.916667,6
10,TNAT1A,DNA,609,171,0.749129,0.014387,0.330882,58231,340.532,181.5610,86.0760,267.6370,133.0,0.777778,TNAT1A#DNA,DNA,"0,0",0.888889,0
181,ATCOPIA44,LTR/Copia,5194,12,0.740917,0.051360,0.252663,15503,1291.920,1819.9200,2098.8300,3918.7500,11.0,0.916667,ATCOPIA44#LTR/Copia,LTR,"5,5",0.958333,5
94,VANDAL12,DNA/DTM,11974,33,0.707000,0.020019,0.196031,14231,431.242,4852.1800,6690.3300,11542.5000,33.0,1.000000,VANDAL12#DNA/DTM,DNA,"8,9",1.000000,0
124,ATTIRTA1,DNA/DTT,265,114,0.796789,0.015528,0.239349,23319,204.553,12.5439,44.2456,56.7895,54.0,0.473684,ATTIRTA1#DNA/Tc1,DNA,"2,2",0.736842,0
157,ATCOPIA26,LTR/Copia,4875,11,0.708545,0.033632,0.175129,8576,779.636,1123.1800,2976.3600,4099.5500,10.0,0.909091,ATCOPIA26#LTR/Copia,LTR,"5,5",0.954546,11
129,ATLINE1_4,LINE/L1,6700,76,0.666053,0.014918,0.142107,49440,650.526,4186.5900,1861.2400,6047.8300,76.0,1.000000,ATLINE1_4#LINE/L1,LINE,"5,20",1.000000,0


In [28]:
te_sum[(te_sum['superfamily'] == "RC/Helitron")]

,family,superfamily,full_length_bp,copynumber,mean_identity,mean_indel,total_length,mean_length,mean_headcut,mean_tailcut,mean_cut,fragment_copy,fragment_ratio,original_family,subclass
282,ATREP1,RC/Helitron,888,151,0.860543,0.299144,85735,567.781,144.1060,162.7280,306.8340,77.0,0.509934,ATREP1#RC/Helitron,RC
283,ATREP10,RC/Helitron,899,61,0.859295,0.297223,26775,438.934,134.0000,371.9020,505.9020,43.0,0.704918,ATREP10#RC/Helitron,RC
284,ATREP10A,RC/Helitron,1380,122,0.784557,0.284708,36067,295.631,485.2210,590.1890,1075.4100,117.0,0.959016,ATREP10A#RC/Helitron,RC
285,ATREP10B,RC/Helitron,1821,103,0.769534,0.303920,41492,402.835,353.9710,977.2140,1331.1800,101.0,0.980583,ATREP10B#RC/Helitron,RC
286,ATREP10C,RC/Helitron,653,170,0.758724,0.277863,43579,256.347,169.2240,271.8880,441.1120,159.0,0.935294,ATREP10C#RC/Helitron,RC
287,ATREP10D,RC/Helitron,777,102,0.794098,0.289073,32443,318.069,221.0780,292.1760,513.2550,85.0,0.833333,ATREP10D#RC/Helitron,RC
288,ATREP11,RC/Helitron,1053,175,0.835560,0.283184,89699,512.566,258.6230,286.3600,544.9830,139.0,0.794286,ATREP11#RC/Helitron,RC
289,ATREP11A,RC/Helitron,1003,47,0.870191,0.215861,20727,441.000,259.3620,268.3400,527.7020,31.0,0.659574,ATREP11A#RC/Helitron,RC
290,ATREP12,RC/Helitron,1342,26,0.836462,0.340925,12482,480.077,73.3462,764.0380,837.3850,19.0,0.730769,ATREP12#RC/Helitron,RC
291,ATREP13,RC/Helitron,648,146,0.798842,0.337378,48928,335.123,65.8151,237.4930,303.3080,102.0,0.698630,ATREP13#RC/Helitron,RC


In [ ]:
df[['width’, 'length’, 'species']]